
# JSON Standardization and Flattening
This notebook is responsible for standardizing and flattening JSON files. It reads the raw JSON data from the landing zone, applies a schema for validation, flattens nested structures, and then writes the transformed data as Delta Parquet files.

Key Features:
- Reads JSON files from the landing zone.
- Applies a predefined schema for data validation (ensure the schema file is available at landing/schemachecks/[datasetidentifier]/[datasetidentifier]_schema.json).
- Flattens nested JSON structures for easier querying and analysis.
- Saves the processed data as Delta Parquet files for efficient storage and querying.

## Widget Initialization and Configuration

### triton__flow_plans

In [ ]:
dbutils.widgets.removeAll()

# Source and Destination Storage Accounts
dbutils.widgets.text("SourceStorageAccount", "dplandingstoragetest", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "dpuniformstoragetest", "Destination Storage Account")

# Source container and dataset identifier
dbutils.widgets.text("SourceContainer", "landing", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "triton__flow_plans", "Source Datasetidentifier")

# Source file and key columns
dbutils.widgets.text("SourceFileName", "*", "Source File Name")
dbutils.widgets.text("KeyColumns", "Guid", "Key Columns")

# Feedback column and flattening option
dbutils.widgets.text("FeedbackColumn", "EventTimestamp", "Feedback Column")
dbutils.widgets.text("DepthLevel", "", "Depth Level")
dbutils.widgets.text("SchemaFolderName", "schemachecks", "Schema Folder Name")

## Setup

### Package Installation and Management

In [ ]:
# Install necessary packages
# Ensure to use the specific versions needed.

# Uninstall the existing package version (uncomment if a previous version is present).
# %pip uninstall databricks-custom-utils -y

# Install the package from the GitHub repository's specified branch.
%pip install git+https://github.com/xazms/utils-databricks.git@v0.6.7 sqlparse openpyxl

### Initialize notebook and get parameters

In [ ]:
from custom_utils.logging.logger import Logger
from custom_utils.config.config import Config

# Instantiate the logger class
logger = Logger(debug=True)

# Initialize the Config object
config = Config.initialize(dbutils=dbutils, logger=logger, debug=True)

# Unpack parameters into the current scope
config.unpack(globals())

### Verify paths and files

In [ ]:
from custom_utils.validation.validation import Validator

# Initialize the Validator class with the provided configuration and logger
validator = Validator(config, logger=logger, debug=True)

# Perform validation of schema and source paths, returning validated file folder paths and file type.
schema_file_path, data_file_path, matched_data_files, data_file_type = validator.verify_paths_and_files()


## Processing Workflow

### Flattening and Processing Nested JSON Data

In [ ]:
from pyspark.sql.functions import col
from custom_utils.transformations.dataframe import DataFrameTransformer

# Initialize the DataFrameTransformer class
transformer = DataFrameTransformer(config=config, logger=logger, dbutils=dbutils, debug=True)

# Call the process_and_flatten_data method using the transformer instance
df_initial, df_flattened = transformer.process_and_flatten_data(
    schema_file_path=schema_file_path,
    data_file_path=data_file_path,
    file_type=data_file_type,
    depth_level=depth_level,
    sheet_name=sheet_name
)

# Display the flattened DataFrame to verify the result (useful in Databricks or Jupyter)
display(df_flattened)

## Quality check 

### Abort if duplicates exists in new data

In [ ]:
from custom_utils.quality.quality import DataQualityManager

# Initialize the DataQualityManager class
quality_manager = DataQualityManager(logger=logger, debug=True)

# Describe available quality checks
quality_manager.describe_available_checks()

# Executing Data Quality Checks on Flattened DataFrame
cleaned_data_view = quality_manager.perform_data_quality_checks(
    spark=spark,
    df=df_flattened,
    key_columns=key_columns,
    order_by=key_columns,
    join_column=key_columns,
    columns_to_exclude=["input_file_name"],
    use_python=False # Set to True to use Python DataFrame operations; False for SQL
)

## Write

### Unified Data Storage Management: Table Creation and Data Merging

In [ ]:
from custom_utils.catalog.catalog_utils import DataStorageManager

# Initialize the DataStorageManager class
storage_manager = DataStorageManager(logger=logger, debug=True)

storage_manager.manage_data_operation(
    spark=spark,
    dbutils=dbutils,
    cleaned_data_view=cleaned_data_view,  # The view you want to merge
    key_columns=key_columns,  # Key columns to use for merging
    destination_folder_path=destination_folder_path,  # Destination folder path
    destination_environment=destination_environment,  # The database name
    source_datasetidentifier=source_datasetidentifier,  # The table name
    use_python=False  # Set to True to use Python DataFrame operations; False for SQL
)

## Finishing

### Return period (from_datetime, to_datetime) covered by data read

In [ ]:
# Run the main function to generate feedback timestamps
notebook_output = storage_manager.generate_feedback_timestamps(
    spark=spark, 
    view_name=cleaned_data_view, 
    feedback_column=None,
    key_columns=key_columns
)

## Exit the notebook

In [ ]:
# Use dbutils to exit the notebook
dbutils.notebook.exit(notebook_output)